In [1]:
import csv
import numpy as np
import pandas as pd
import numpy as np
import os
import re 
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.polynomial.polynomial import Polynomial
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


C:\Users\james\AppData\Local\Temp\ipykernel_23404\2559101972.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
file = open('flexural/ES100_Trevor_Flexural_Data_1_1.csv')
type(file)

_io.TextIOWrapper

In [11]:
csvreader = csv.reader(file)
header = []
header = next(csvreader)
header

['Specimen text inputs : Specimen text input  1', '', '']

In [12]:
id_dict = {'Basalt 0deg': [1, 2], 'Hemp 45deg': [11, 12], 'Flax 45deg': [7,8], 'Basalt Hemp 0deg': [13, 14], 'Hemp 0deg': [9, 10], 'Flax 0deg': [5,6], 'Basalt Flax 0deg': [15, 16], 'Basalt 45deg': [3,4]}

In [15]:
import pandas as pd
import numpy as np
import os
import re  # Import regular expressions for finding the id in the file name

def get_name_for_id(id_num):
    for name, ids in id_dict.items():
        if id_num in ids:
            return name
    return None 

def convert_to_numeric(df, column_names):
    for column in column_names:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

def average_n_values(df, n):
    avg_df = df.groupby(np.arange(len(df)) // n).mean()
    avg_df['Time'] = avg_df['Time'].round().astype(int)
    return avg_df

folder_path = 'flexural'
n = 50 
dfs = []

names = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Attempt to automatically detect the start of the data
        with open(file_path, 'r') as file:
            for i, line in enumerate(file):
                if line.strip().split(',')[0] == "Time":
                    header_line = i
                    print('start of data found')
                    break

        # Extract the id number from the file name using regular expressions
        match = re.search(r"ES100_Trevor_Flexural_Data_(\d+)_1.csv", file_name)
        if match:
            id_num = int(match.group(1))
            name = get_name_for_id(id_num)
            if name:
                file_name_modified = file_name.replace(f"ES100_Trevor_Flexural_Data_{id_num}_1", f"{name}_{names.count(name) + 1}")
                names.append(name)
            else:
                file_name_modified = file_name
        else:
            file_name_modified = file_name

        df = pd.read_csv(file_path, skiprows=header_line)

        df = convert_to_numeric(df, ['Time', 'Displacement', 'Force'])
        print(df)
        df_avg = average_n_values(df, n)

        df_avg['File_ID'] = os.path.splitext(file_name_modified)[0]

        df_avg['Material_ID'] = file_name.replace(f"ES100_Trevor_Flexural_Data_{id_num}_1.csv", f"{name}")

        dfs.append(df_avg)

combined_df = pd.concat(dfs, ignore_index=True)

filtered_dfs = []

# Group combined_df by Material_ID
for material_id, group in combined_df.groupby('Material_ID'):
    # Calculate mean force for each File_ID within the current material_id group
    file_mean_force = group.groupby('File_ID')['Force'].mean()
    
    # Calculate the deviation of each File_ID's mean force from the material's mean force
    material_mean_force = file_mean_force.mean()
    file_force_deviation = file_mean_force.apply(lambda x: abs(x - material_mean_force))
    
    # Sort by deviation to get the File_IDs in order of their deviation
    sorted_files_by_deviation = file_force_deviation.sort_values(ascending=False)
    
    # Identify the File_IDs to keep (skip the one with the most outlying force)
    files_to_keep = sorted_files_by_deviation.iloc[1:5].index.tolist()
    
    # Filter the group to only include rows with File_IDs in files_to_keep
    filtered_group = group[group['File_ID'].isin(files_to_keep)]
    
    # Append the filtered group to the list of DataFrames
    filtered_dfs.append(filtered_group)

# Concatenate all filtered DataFrames
filtered_combined_df = pd.concat(filtered_dfs, ignore_index=True)

# Save the filtered combined dataframe
filtered_combined_df.to_csv('combined_flexural.csv', index=False)

print("Processing complete. The combined CSV is saved as 'combined_flexural.csv'.")


start of data found
        Time  Displacement   Force
0        NaN           NaN     NaN
1      0.000        0.4313  0.0202
2      0.020        0.4366  0.0210
3      0.040        0.4392  0.0217
4      0.060        0.4398  0.0218
...      ...           ...     ...
3599  71.960        6.4268  0.1321
3600  71.980        6.4285  0.1318
3601  72.000        6.4302  0.1315
3602  72.020        6.4318  0.1316
3603  72.032        6.4329  0.1317

[3604 rows x 3 columns]
start of data found
         Time  Displacement   Force
0         NaN           NaN     NaN
1       0.000        0.8652  0.0202
2       0.020        0.8707  0.0209
3       0.040        0.8731  0.0217
4       0.060        0.8738  0.0224
...       ...           ...     ...
5066  101.300        9.3059  0.0117
5067  101.320        9.3076  0.0117
5068  101.340        9.3093  0.0117
5069  101.360        9.3108  0.0123
5070  101.366        9.3113  0.0124

[5071 rows x 3 columns]
start of data found
        Time  Displacement   Force
0  

In [18]:
import csv
import re

# Placeholder function for any transformation you want to apply to the Displacement values
def transform_displacement(value):
    # Example transformation; replace with your actual logic
    return float(value) * 100

# Process the file
input_file_path = 'flexural/ES100_Trevor_Flexural_Data_1_1.csv'
output_file_path = 'flexural/ES100_Trevor_Flexural_Data_1_1_1.csv'

with open(input_file_path, 'r') as infile, open(output_file_path, 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    for row in reader:
        # Add quotation marks for non-empty, non-header rows in the data section
        if len(row) == 3 and row[0] != 'Time' and row[0] and any(row):
            # Check if the row is numeric, which indicates a data row
            if re.match(r'^\d+(\.\d+)?$', row[0]):
                row[0] = f'"{float(row[0]):.4f}"'  # Time with quotes and formatted
                row[1] = f'"{transform_displacement(row[1]):.4f}"'  # Displacement with transformation
                row[2] = f'"{float(row[2]):.4f}"'  # Force with quotes
            else:
                # For non-numeric rows that should be quoted
                row = [f'"{item}"' if item else '""' for item in row]
        writer.writerow(row)

print("File has been processed and saved.")


File has been processed and saved.
